In [1]:
import os
import pandas as pd
import numpy as np
import tomotopy as tp
from tomotopy.utils import Corpus
from sklearn.model_selection import KFold
from local.caching import load, save

df: pd.DataFrame = load('biocyc_metabolite_usage', alt_workspace="../prep/")
df.head()

recovering & decompressing cached data from [../prep/cache/biocyc_metabolite_usage.pkl.gz]


,i_L-CYSTATHIONINE,i_UTP,i_CPD-12575,i_PPI,i_GLC-1-P,i_PROTON,i_ACP,i_MALONYL-COA,i_Hydroxymethylbilane-Synthase-ES2,i_PORPHOBILINOGEN,...,c_CPLX66-208,c_CPLX66-201,c_ENSG00000157933-MONOMER,c_O-Acyl-L-Carnitines,c_I-SMAD,c_UDP-GLUCURONATE,c_CPD-12575,c_NOREPINEPHRINE,c_Nucleoside-Monophosphates,c_3-4-DIHYDROXYBENZOATE
gcf_014653195cyc.pgdb,3,8,3,12,4,333,5,5,1,5,...,0,0,0,0,0,0,0,0,0,0
gcf_000359505cyc.pgdb,1,8,5,11,6,422,3,6,1,5,...,0,0,0,0,0,0,0,0,0,0
gcf_003952285cyc.pgdb,1,8,5,13,4,515,40,13,0,1,...,0,0,0,0,0,0,0,0,0,0
gcf_000744815cyc.pgdb,4,5,13,14,6,342,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0
gcf_000701505cyc.pgdb,1,5,4,9,7,278,4,7,0,1,...,0,0,0,0,0,0,0,0,0,0


In [2]:
from concurrent.futures import ProcessPoolExecutor as Exe

folder = "./cache/corpus"
if not os.path.exists(folder): os.makedirs(folder)
def make_corp(name: str, indexes: np.ndarray):
    save_name = f'{folder}/{name}'
    if os.path.exists(save_name): return

    df: pd.DataFrame = load('biocyc_metabolite_usage', alt_workspace="../prep/")
    corp = Corpus()
    columns = df.columns
    label_len = max([len(n) for n in columns])

    selected = df.iloc[indexes]
    for i, (_, row) in enumerate(selected.iterrows()):
        # if i % 16 == 0: print(f"{i+1} of {len(selected)}", " "*25, end='\r')
        doc = np.empty(shape=(row.sum(),), dtype=f'<U{label_len}')
        doc_i = 0
        for j, c in enumerate(row):
            if c == 0: continue
            for _ in range(c):
                doc[doc_i] = columns[j]
                doc_i += 1
        corp.add_doc(doc)

    corp.save(save_name)
    return name

np.random.seed(12345)
indicies = np.array(range(len(df)), dtype=int)
k_min, k_max, k_step = 10, 120, 10
args = []
for k in [5]+[i for i in range(k_min, k_max+1, k_step)]:
    np.random.shuffle(indicies)
    kf = KFold(n_splits=5)
    for j, (train, test) in enumerate(kf.split(indicies)):
        # print(f'{k}-{j}', ' '*25)
        train_i, test_i = indicies[train], indicies[test]
        # make_corp(f'k{k}_{j}_train', train_i)
        # make_corp(f'k{k}_{j}_test', test_i)

        args.append((f'k{k}_{j}_train', train_i))
        args.append((f'k{k}_{j}_test', test_i))

with Exe(max_workers=2) as exe:
    for result in exe.map(make_corp, [n for n, ilst in args], [ilst for n, ilst in args], chunksize=1):
        if result is None: continue
        print(f"{result} completed")

recovering & decompressing cached data from [../prep/cache/biocyc_metabolite_usage.pkl.gz]recovering & decompressing cached data from [../prep/cache/biocyc_metabolite_usage.pkl.gz]

k60_0_train completed
k60_0_test completed
